# Model Tuning

This notebook will demonstate how to automatically tune model hyperparameters for prediction accuracy.

## Set-Up
We will begin with some imports and then generate some training data using the simple cart-pole benchmark.

In [3]:
import os
os.chdir('/home/randomgraph/baoyul2/meta/autompc')
import autompc as ampc
import numpy as np

from autompc.benchmarks import CartpoleSwingupBenchmark, CartpoleSwingupV2Benchmark

# benchmark = CartpoleSwingupBenchmark()
benchmark = CartpoleSwingupV2Benchmark()

system = benchmark.system
trajs = benchmark.gen_trajs(seed=100, n_trajs=100, traj_len=200)

## Auto-Tuning

By default, AutoMPC's `ModelTuner` will auto-select from all available models to fit the trajectories as best as possible. The tuner by default will run for 10 iterations, but for real problems you will want to run for many, many more iterations (100s).

In [4]:
from autompc.tuning import ModelTuner

tuner = ModelTuner(system,trajs,verbose=1)
print("Selecting from models",",".join(model.name for model in tuner.model.models))
tuned_model,tune_result = tuner.run(n_iters=2)

print("Selected model:",tuned_model.name)
print("Final cross-validated RMSE score:",tune_result.inc_costs[-1])

Cuda is used for GPytorch
Selecting from models MLP,ARX,Koopman,SINDy,ApproximateGPModel
Evaluating Cfg:
Configuration(values={
  'MLP:batchnorm': False,
  'MLP:hidden_size_1': 128,
  'MLP:hidden_size_2': 128,
  'MLP:lr': 0.001,
  'MLP:n_hidden_layers': '2',
  'MLP:nonlintype': 'relu',
  'model': 'MLP',
})

Seed 0 budget 0.0
100%|██████████| 200/200 [00:55<00:00,  3.63it/s]


max_iter must be positive
Traceback (most recent call last):
  File "/home/randomgraph/anaconda3/envs/meta/lib/python3.9/site-packages/smac/tae/execute_func.py", line 217, in run
    rval = self._call_ta(self._ta, config, obj_kwargs)
  File "/home/randomgraph/anaconda3/envs/meta/lib/python3.9/site-packages/smac/tae/execute_func.py", line 314, in _call_ta
    return obj(config, **obj_kwargs)
  File "/home/randomgraph/baoyul2/meta/autompc/autompc/tuning/model_tuner.py", line 120, in _evaluate
    value = self.evaluator(self.model)
  File "/home/randomgraph/baoyul2/meta/autompc/autompc/tuning/model_evaluator.py", line 160, in __call__
    m.train(train)
  File "/home/randomgraph/baoyul2/meta/autompc/autompc/sysid/autoselect.py", line 68, in train
    self.selected_model.train(trajs)
  File "/home/randomgraph/baoyul2/meta/autompc/autompc/sysid/sindy.py", line 160, in train
    self._init_model()
  File "/home/randomgraph/baoyul2/meta/autompc/autompc/sysid/sindy.py", line 156, in _init_mode

Model Score  0.017887528491952753
Evaluating Cfg:
Configuration(values={
  'SINDy:poly_basis': 'true',
  'SINDy:poly_cross_terms': 'false',
  'SINDy:poly_degree': 3,
  'SINDy:threshold': 0.03240796443598578,
  'SINDy:time_mode': 'discrete',
  'SINDy:trig_basis': 'true',
  'SINDy:trig_freq': 7,
  'SINDy:trig_interaction': 'false',
  'model': 'SINDy',
})

Seed 0 budget 0.0
100%|██████████| 200/200 [01:23<00:00,  2.40it/s]
Selected model: MLP
Final cross-validated RMSE score: 0.017887528491952753


: 

You can customize the behavior of tuning by specifying which evaluation strategy we wish to use.  Possible options include splitting method (holdout vs cross-validation), which horizon to measure predictions upon, and what scoring metric to use.  By default, ModelTuner uses 3-fold cross-validation and one-step RMSE.

Here's an example of customizing the evaluator to use with 10\% holdout and the RMSE metric with a 5-step prediction horizon.

In [ ]:
tuner = ModelTuner(system,trajs,eval_holdout=0.1,eval_folds=1,eval_metric='rmse',eval_horizon=5)

To customize behavior even further, we can use a `ModelEvaluator` class, which has various subclasses.  As an example, the HoldoutModelEvaluator is specified here.  The `evaluator` keyword to ModelTuner will specify an evaluator that overrides the default keyword arguments.

In [ ]:
from autompc.tuning import HoldoutModelEvaluator

evaluator = HoldoutModelEvaluator(trajs, metric="rmse", holdout_prop=0.1,
                                  rng=np.random.default_rng(100), horizon=20)
tuner = ModelTuner(system,trajs,evaluator=evaluator)

## Specifying the Model Class

In some cases we know which model class we wish to tune, and ModelTuner can also accept a specified class.  For example, we can consider the MLP model. Here we'll perform a much longer tuning run, so let this run for a few hours...

**Alternatively, you can save/load the tuning data from a prior run by setting `dump=False` in the following cell, and skipping the tuning altogether.**

In [ ]:
from autompc.sysid import MLP

tuner = ModelTuner(system,trajs,MLP(system),verbose=True)
tuned_model, tune_result = tuner.run(n_iters=200,rng=np.random.default_rng(200))

In [ ]:
import pickle

#To dump tuning result, turn to True.  To load it, turn to False
DUMP = False
if DUMP:
    with open('tuned_mlp_model.pkl','wb') as f:
        pickle.dump(tuned_model,f)
    with open('mlp_model_tuning_result.pkl','wb') as f:
        pickle.dump(tune_result,f)
else:
    with open('tuned_mlp_model.pkl','rb') as f:
        tuned_model = pickle.load(f)
    with open('mlp_model_tuning_result.pkl','rb') as f:
        tune_result = pickle.load(f)

Let's check what configuration was selected by the tuner.

In [ ]:
tune_result.inc_cfg

We see that the tuner selected a 2-layer MLP with `tanh` activations. 

Before we move on, let's note that another option is to specify a set of model classes to use. To do so we can use the AutoSelectModel class as the model as follows:

In [ ]:
from autompc.sysid import SINDy
from autompc.sysid import AutoSelectModel

selector = AutoSelectModel(system,[MLP(system),SINDy(system)])
tuner = ModelTuner(system,trajs,model=selector)

## Visualizing the Results

We can now visualize the tuning results.  First, we will plot the tuning curve.  This shows the holdout set performance of the best-known model at different points over the course of the tuning process.

In [ ]:
%matplotlib inline

from autompc.graphs import plot_tuning_curve,plot_tuning_correlations
import matplotlib.pyplot as plt

plot_tuning_curve(tune_result)
plt.title("Cart-Pole Tuning Curve")
plt.show()

To further study the tuning, we can examine how the cost correlates with various hyperparameter settings using the `plot_tuning_correlations` function.

In [ ]:
fig,(ax1,ax2,ax3)=plt.subplots(1,3,figsize=(15,4))
plot_tuning_correlations(tune_result,'lr',ax=ax1)
plot_tuning_correlations(tune_result,'n_hidden_layers',ax=ax2)
plot_tuning_correlations(tune_result,'nonlintype',ax=ax3)
ax1.set_ylim(0,1.5)
ax2.set_ylim(0,1.5)
ax3.set_ylim(0,3.0)
ax3.set_title('nonlintype')
plt.plot()

Next, we can compare the performance of our tuned model to the default MLP configuration.  We will generate a fresh dataset for testing and compare over multiple prediction horizons.  For more details on how to do this comparison, see [2. Models].

In [ ]:
untuned_model = MLP(system)
untuned_model.train(trajs)

testing_set = benchmark.gen_trajs(seed=101, n_trajs=100, traj_len=200)

from autompc.graphs.kstep_graph import KstepPredAccGraph

graph = KstepPredAccGraph(system, testing_set, kmax=20, metric="rmse")
graph.add_model(untuned_model, "Untuned MLP")
graph.add_model(tuned_model, "Tuned Model (MLP)")

fig = plt.figure()
ax = fig.gca()
graph(fig, ax)
ax.set_title("Model prediction accuracy on test set")
plt.show()

As we can see, the tuned model outperforms the untuned model on the unseen dataset at all prediction horizons.

In [ ]:
graph = KstepPredAccGraph(system, trajs, kmax=20, metric="rmse")
graph.add_model(untuned_model, "Untuned MLP")
graph.add_model(tuned_model, "Tuned Model (MLP)")

fig = plt.figure()
ax = fig.gca()
graph(fig, ax)
ax.set_title("Model prediction accuracy on training set")
plt.show()